In [2]:
from __future__ import print_function
import sys
import re
import numpy as np

from numpy import dot
from numpy.linalg import norm
from pyspark import SparkContext
from operator import add
import warnings
warnings.filterwarnings("ignore")
from sklearn.metrics import f1_score
from sklearn.metrics import confusion_matrix
from pyspark.mllib.classification import LogisticRegressionWithLBFGS
from pyspark.mllib.regression import LabeledPoint
from sklearn.metrics import f1_score
from sklearn.metrics import confusion_matrix
from time import time

Dimention=20000
smallTrainFile='SmallTrainingData.txt'
TestFile='TestingData.txt'

In [3]:
def freqArray (listOfIndices, numberofwords):
    returnVal = np.zeros (Dimention)
    for index in listOfIndices:
        returnVal[index] = returnVal[index] + 1
    returnVal = np.divide(returnVal, numberofwords)
    return returnVal


In [4]:
sc.stop()
# load training set
start_time = time()
sc = SparkContext(appName="SVM")

d_corpus = sc.textFile(smallTrainFile)
t_d_corpus = sc.textFile(TestFile)


numberOfDocs =d_corpus.count()

d_keyAndText = d_corpus.map(lambda x : (x[x.index('id="') + 4 : x.index('" url=')], x[x.index('">') + 2:][:-6]))
regex = re.compile('[^a-zA-Z]')
d_keyAndListOfWords = d_keyAndText.map(lambda x : (str(x[0]), regex.sub(' ', x[1]).lower().split()))

allWords = d_keyAndListOfWords.flatMap(lambda x: ((i,1) for i in x[1]))

allCounts = allWords.reduceByKey(lambda x,y: x+y)
allCounts.cache()
topWords = allCounts.top(Dimention, lambda x : x[1])

topWordsK = sc.parallelize(range(Dimention))
dictionary = topWordsK.map (lambda x : (topWords[x][0], x))

allWordsWithDocID = d_keyAndListOfWords.flatMap(lambda x: ((j, x[0]) for j in x[1]))

allDictionaryWords = dictionary.join (allWordsWithDocID)


justDocAndPos = allDictionaryWords.map (lambda x: (x[1][1], x[1][0]))

allDictionaryWordsInEachDoc = justDocAndPos.groupByKey()

allDocsAsNumpyArrays = allDictionaryWordsInEachDoc.map(lambda x: (x[0], freqArray(x[1],len(x[1]))))

allDocsAsNumpyArrays.cache()
allDocsAsNumpyArrays.take(3)

[('AU466',
  array([0.10181311, 0.06229661, 0.0125523 , ..., 0.        , 0.        ,
         0.        ])),
 ('AU1356',
  array([0.08316993, 0.04428105, 0.02320261, ..., 0.        , 0.        ,
         0.        ])),
 ('19264299',
  array([0.076326  , 0.03751617, 0.04915912, ..., 0.        , 0.        ,
         0.        ]))]

In [5]:
# load test set
t_numberOfDocs =t_d_corpus.count()

t_d_keyAndText = t_d_corpus.map(lambda x : (x[x.index('id="') + 4 : x.index('" url=')], x[x.index('">') + 2:][:-6]))
regex = re.compile('[^a-zA-Z]')
t_d_keyAndListOfWords = t_d_keyAndText.map(lambda x : (str(x[0]), regex.sub(' ', x[1]).lower().split()))

t_allWords = t_d_keyAndListOfWords.flatMap(lambda x: ((i,1) for i in x[1]))

t_allCounts = t_allWords.reduceByKey(lambda x,y: x+y)
t_allCounts.cache()
t_topWords = t_allCounts.top(Dimention, lambda x : x[1])

topWordsK = sc.parallelize(range(Dimention))
t_dictionary = topWordsK.map (lambda x : (t_topWords[x][0], x))

t_allWordsWithDocID = t_d_keyAndListOfWords.flatMap(lambda x: ((j, x[0]) for j in x[1]))

t_allDictionaryWords = t_dictionary.join (t_allWordsWithDocID)


t_justDocAndPos = t_allDictionaryWords.map (lambda x: (x[1][1], x[1][0]))

t_allDictionaryWordsInEachDoc = t_justDocAndPos.groupByKey()

t_allDocsAsNumpyArrays = t_allDictionaryWordsInEachDoc.map(lambda x: (x[0], freqArray(x[1],len(x[1]))))
t_allDocsAsNumpyArrays.cache()
print(t_allDocsAsNumpyArrays .take(3))
load_data_duration=time() - start_time

[('12027634', array([0.05555556, 0.03262787, 0.03880071, ..., 0.        , 0.        ,
       0.        ])), ('12957200', array([0.05494505, 0.01775148, 0.01690617, ..., 0.        , 0.        ,
       0.        ])), ('12987837', array([0.09176916, 0.02081362, 0.02932829, ..., 0.        , 0.        ,
       0.        ]))]


In [13]:
def get_label(x):
    if 'AU' in x[0]:
        return 1
    else:
        return -1
    
def get_vector(x):
    return (np.max([0,(1-x[0]*np.dot(w_current,x[1]))]),np.zeros(Dimention) if (1-x[0]*np.dot(w_current,x[1])) < 0 else (-x[0]*x[1]))

In [30]:
# SVM
w_current=np.random.normal(0,10,Dimention)
learningRate=0.5
num_iteration = 300
oldCost=0
n=numberOfDocs
lamda = 100

allDocsAsNumpyArrays.cache()
Y_X_RDD=allDocsAsNumpyArrays.map(lambda x: ((get_label(x), x[1])))

train_start_time = time()
for i in range(num_iteration):
   
    
    Gradient=Y_X_RDD.map(get_vector)
    GradientSum = Gradient.reduce(lambda x,y: ((x[0] + y[0]),(x[1] + y[1])))
    
    # calculate cost:
    cost = (1/(2*(n*lamda)))*np.dot(w_current,w_current)+(GradientSum[0]/n)
    
    oldp = np.sqrt(abs(np.dot(w_current,w_current)))
    temp=w_current
       
    w_current= w_current-learningRate*(((1/(n*lamda))*w_current)+(GradientSum[1]/n))
    
    p = np.sqrt(abs(np.dot(temp,w_current)))

    #Stop if the cost is not descreasing 
    if abs(oldp-p) <= 0.0005:
        break

    if(cost<oldCost):
        learningRate=learningRate*1.05
        oldCost = cost

    if(cost>oldCost):
        learningRate=learningRate*0.5
        oldCost = cost
        

    print("Iteration No.=", i , " Cost=", cost) 
    print('w current:',w_current)
print('finish')
train_duration= time()-train_start_time
    
   

Iteration No.= 0  Cost= 4.31499984991981
w current: [ -4.14022246 -11.48790401   7.62074657 ...  12.58197712  -2.8081085
   6.08113861]
Iteration No.= 1  Cost= 4.309853522693819
w current: [ -4.15846381 -11.4964027    7.61296492 ...  12.58196628  -2.80810786
   6.08113349]
Iteration No.= 2  Cost= 4.30728036562213
w current: [ -4.17761721 -11.50532632   7.60479419 ...  12.58195489  -2.80810719
   6.08112811]
Iteration No.= 3  Cost= 4.304578554670687
w current: [ -4.19772827 -11.51469612   7.59621494 ...  12.58194293  -2.80810649
   6.08112246]
Iteration No.= 4  Cost= 4.301742250469342
w current: [ -4.21883651 -11.52453021   7.58721066 ...  12.58193037  -2.80810574
   6.08111653]
Iteration No.= 5  Cost= 4.298767370884133
w current: [ -4.24098513 -11.53484621   7.5777599  ...  12.58191719  -2.80810497
   6.08111031]
Iteration No.= 6  Cost= 4.295645990163383
w current: [ -4.26423287 -11.54567549   7.56783914 ...  12.58190334  -2.80810415
   6.08110377]
Iteration No.= 7  Cost= 4.29237059932

Iteration No.= 60  Cost= 3.579428368070599
w current: [ -9.97254798 -14.19563553   5.07661061 ...  12.57816326  -2.8078535
   6.07933572]
Iteration No.= 61  Cost= 3.5518144562049074
w current: [-10.22511409 -14.3121659    4.96176499 ...  12.57796229  -2.80783344
   6.07924185]
Iteration No.= 62  Cost= 3.5242694934565257
w current: [-10.48358328 -14.43107141   4.84376347 ...  12.57775128  -2.80781237
   6.07914328]
Iteration No.= 63  Cost= 3.496835455969797
w current: [-10.74434041 -14.55121348   4.72368065 ...  12.57752972  -2.80779025
   6.07904027]
Iteration No.= 64  Cost= 3.47000340717186
w current: [-11.00820927 -14.67271683   4.60123301 ...  12.57729709  -2.80776703
   6.07893212]
Iteration No.= 65  Cost= 3.4436749119753856
w current: [-11.27519279 -14.79569766   4.47639542 ...  12.57705411  -2.80774264
   6.07881855]
Iteration No.= 66  Cost= 3.4178343391641377
w current: [-11.54473516 -14.91959372   4.34921453 ...  12.57679898  -2.80771704
   6.07869931]
Iteration No.= 67  Cost= 

Iteration No.= 119  Cost= 2.9564474687791424
w current: [-21.04065962 -19.36707584  -1.86920638 ...  12.51991439  -2.79529087
   6.0514901 ]
Iteration No.= 120  Cost= 2.954653264732838
w current: [-21.07328031 -19.38538821  -1.9614768  ...  12.51689265  -2.79461622
   6.05002954]
Iteration No.= 121  Cost= 2.952820927877611
w current: [-21.09831237 -19.40055296  -2.05338685 ...  12.51372059  -2.793908
   6.04849633]
Iteration No.= 122  Cost= 2.9509599184651183
w current: [-21.1149886  -19.4113161   -2.14476938 ...  12.51039077  -2.79316456
   6.04688686]
Iteration No.= 123  Cost= 2.949050460644617
w current: [-21.12076328 -19.41620319  -2.23262525 ...  12.50689539  -2.79238415
   6.04519737]
Iteration No.= 124  Cost= 2.9470946921230943
w current: [-21.12214551 -19.41976324  -2.3217441  ...  12.50322626  -2.79156496
   6.0434239 ]
Iteration No.= 125  Cost= 2.9450704238025582
w current: [-21.11252738 -19.41679152  -2.40660533 ...  12.49937481  -2.79070505
   6.04156231]
Iteration No.= 126

Iteration No.= 178  Cost= 2.5042915228935767
w current: [ -7.0093216  -11.98582143 -13.01459437 ...  11.54422437  -2.57745093
   5.57989114]
Iteration No.= 179  Cost= 2.483025356081996
w current: [ -6.84259528 -11.7915807  -13.59210199 ...  11.49465777  -2.56638432
   5.55593317]
Iteration No.= 180  Cost= 2.460971378800956
w current: [ -6.05431838 -11.34168921 -13.89618648 ...  11.4428363   -2.55481427
   5.53088531]
Iteration No.= 181  Cost= 2.4380363335778616
w current: [ -5.83080187 -11.12290874 -14.47570057 ...  11.38866907  -2.54272048
   5.50470362]
Iteration No.= 182  Cost= 2.4142188769252377
w current: [ -5.10509101 -10.68988506 -14.82323616 ...  11.3320627   -2.53008211
   5.47734298]
Iteration No.= 183  Cost= 2.3895179285089747
w current: [ -5.0067847  -10.50866539 -15.44819887 ...  11.27292145  -2.51687779
   5.44875711]
Iteration No.= 184  Cost= 2.3639480765735597
w current: [ -4.0672051   -9.99069761 -15.74996978 ...  11.21114722  -2.50308561
   5.41889858]
Iteration No.= 

Iteration No.= 237  Cost= 1.0931712656907822
w current: [ -2.578164    -4.97488977 -30.7184845  ...   7.59611994  -1.73505385
   3.67157819]
Iteration No.= 238  Cost= 1.0820899382950424
w current: [  2.55867669  -2.50096038 -29.36278313 ...   7.53346909  -1.72074357
   3.64129595]
Iteration No.= 239  Cost= 1.0608097642279126
w current: [ -6.05724127  -6.55431488 -31.90370027 ...   7.46822827  -1.71061971
   3.60976186]
Iteration No.= 240  Cost= 1.0585577121761227
w current: [  0.24646593  -3.54259483 -30.24085906 ...   7.40031865  -1.69506481
   3.5769378 ]
Iteration No.= 241  Cost= 1.021433388930363
w current: [  2.13531219  -2.67119532 -29.75325205 ...   7.32966193  -1.67888068
   3.54278593]
Iteration No.= 242  Cost= 1.0035556561303438
w current: [ -3.68125727  -5.38238548 -31.45446228 ...   7.25618072  -1.6620496
   3.50726885]
Iteration No.= 243  Cost= 0.9933020315832369
w current: [  3.43999331  -1.95619212 -29.58403197 ...   7.17979895  -1.64455413
   3.47034979]
Iteration No.= 

Iteration No.= 296  Cost= 0.46501405303740534
w current: [  1.68088328  -2.2353427  -29.02100036 ...   4.8904884   -1.13752384
   2.36381346]
Iteration No.= 297  Cost= 0.45848330967323647
w current: [ -0.30200827  -3.13254064 -29.40284187 ...   4.85359244  -1.12894187
   2.34597983]
Iteration No.= 298  Cost= 0.45300479849123393
w current: [  2.63844984  -1.69473641 -28.56429628 ...   4.81514396  -1.11999878
   2.32739578]
Iteration No.= 299  Cost= 0.4483627141320836
w current: [ -7.36914707  -6.51939007 -31.15444936 ...   4.77509286  -1.11527202
   2.30803712]
Iteration No.= 300  Cost= 0.4616361868406402
w current: [ -1.22400054  -3.59001554 -29.54743599 ...   4.73338899  -1.10553165
   2.28787959]
Iteration No.= 301  Cost= 0.4337614550229974
w current: [  0.88384685  -2.57661083 -28.96046392 ...   4.71271917  -1.100704
   2.27788885]
Iteration No.= 302  Cost= 0.4271437852047142
w current: [  1.18815727  -2.40483236 -28.82542949 ...   4.69111063  -1.09565711
   2.26744438]
Iteration No

In [23]:
test_start_time=time()
y_ypred = t_allDocsAsNumpyArrays.map(
        lambda x: (1 if 'AU' in x[0] else -1, 1 if np.dot(x[1], w_current) > 0 else -1)).collect()

y = []
ypred = []
counter = 0
for i in y_ypred:
    y.append(i[0])
    ypred.append(i[1])
print('f1 score:', f1_score(y, ypred, average='binary'))
print('confusion matrix:', confusion_matrix(y, ypred))
test_duration=time()-test_start_time
total_duration=time() - start_time
print('load data duration(s):%.4f, train model duration(s):%.4f, test model duration(s):%.4f total duration(s): %.4f'
    %(load_data_duration,train_duration,test_duration,total_duration))

f1 score: 0.01265412070084361
confusion matrix: [[12599  5748]
 [  338    39]]
load data duration(s):77.7909, train model duration(s):1.0607, test model duration(s):1.3196 total duration(s): 1715.2183
